# 01 - BigQuery - Table Data Source
Use BigQuery to load and prepare data for machine learning:

### Video Walkthrough of this notebook:
Includes conversational walkthrough and more explanatory information than the notebook:
<p><center><a href="https://youtu.be/Z5whg20WvS8" target="_blank" rel="noopener noreferrer"><img src="architectures/thumbnails/playbutton/01.png" width="50%"></a></center></p>

### Prerequisites:
-  00 - Environment Setup

### Overview:
-  Setup BigQuery
   -  Create a Dataset
      -  Use BigQuery Python Client
   -  Create Tables
      -  Copy from another Project:Dataset
         -  SQL with BigQuery Jupyter Magic (%%bigquery)
      -  Load data from GCS Bucket
         -  BigQuery Python Client (load_table_from_uri)
   -  Prepare Data For Analysis
      -  Run SQL Queries to prepare Unique ID's and Train/Test Splits

### Resources:
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Download BigQuery Data to Pandas](https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas)
-  [BigQuery Template Notebooks](https://github.com/GoogleCloudPlatform/bigquery-notebooks/tree/main/notebooks/official/template_notebooks)



---
## Source Data

**Overview**

This notebook imports source data for this project into Google BigQuery.  All the remaining notebooks utilize BigQuery as the source and leverage API's native to the machine learning approaches they feature.

In the enviornment setup notebook (00), a BigQuery source table was exported to CSV format in a Cloud Storage Bucket. This notebook, `01 - BigQuery - Table Data Source`, start the machine learning lifecycle by importing a source and preparing it for machine learning.  To customize this series of notebooks change the source referenced here or in notebook `00 - Environment Setup`.

All of these workflows utilize tabular data to fit a supervised learning model: predict a target variable by learning patterns in feature columns.  The type of supervised learning used in these projects is classification: models with a target variable that has multiple discrete classes.  

**The Data**

The source data is exported to Google Cloud Storage in CSV format by the `00 - Environment Setup` notebook.  The BigQuery source table is `bigquery-public-data.ml_datasets.ulb_fraud_detection`.  This is a table of credit card transactions that are classified as fradulant, `Class = 1`, or normal `Class = 0`.  

The data can be researched further at this [Kaggle link](https://www.kaggle.com/mlg-ulb/creditcardfraud).

**Description of the Data**

This is a table of 284,207 credit card transactions classified as fradulant or normal in the column `Class`.  In order protect confidentiality, the original features have been transformed using [principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis) into 28 features named `V1, V2, ... V28` (float).  Two descriptive features are provided without transformation by PCA:
- `Time` (integer) is the seconds elapsed between the transaction and the earliest transaction in the table
- `Amount` (float) is the value of the transaction
>**Quick Note on PCA**

>PCA is an unsupervised learning technique: there is not a target variable.  PCA is commonlly used as a variable/feature reduction technique.  If you have 100 features then you could reduce it to a number p (say 10) projected features.  The choice of this number is a balance of how well it can explain the variance of the full feature space and reducing the number of features.  Each projected feature is orthogonal to each other feature, meaning there is no correlation between these new projected features.

**Preparation of the Data**

This notebook adds two columns to the source data and stores it in a new table with suffix `_prepped`.  
- `transaction_id` (string) a unique id for the row/transaction
- `splits` (string) this divided the tranactions into sets for `TRAIN` (80%), `VALIDATA` (10%), and `TEST` (10%)

---
## Setup

inputs:

In [1]:
PROJECT_ID = "nguyen-demo5"
REGION = 'us-central1'
DATANAME = 'taxi'
NOTEBOOK = '01'

packages:

In [2]:
from google.cloud import bigquery

clients:

In [3]:
bq = bigquery.Client(project = PROJECT_ID)

parameters:

In [4]:
BUCKET = PROJECT_ID

---
## Create Dataset

List BigQuery datasets in the project:

In [5]:
query = f"""
SELECT schema_name
FROM `{PROJECT_ID}.INFORMATION_SCHEMA.SCHEMATA`
"""
bq.query(query = query).to_dataframe()

,schema_name


Create the dataset if missing:

In [6]:
query = f"""
CREATE SCHEMA IF NOT EXISTS `{PROJECT_ID}.{DATANAME}`
OPTIONS(
    location = '{REGION}',
    labels = [('notebook','{NOTEBOOK}')]
)
"""
job = bq.query(query = query)
job.result()

In [7]:
(job.ended-job.started).total_seconds()

0.179

---
## Create Table
- import data from Cloud Storage Bucket
- https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv

In [8]:
destination = bigquery.TableReference.from_string(f"{PROJECT_ID}.{DATANAME}.{DATANAME}")
job_config = bigquery.LoadJobConfig(
    write_disposition = 'WRITE_TRUNCATE',
    source_format = bigquery.SourceFormat.CSV,
    autodetect = True,
    labels = {'notebook':f'{NOTEBOOK}'}
)
job = bq.load_table_from_uri(f"gs://{BUCKET}/{DATANAME}/data/{DATANAME}_*.csv", destination, job_config = job_config)
job.result()

LoadJob<project=nguyen-demo5, location=us-central1, id=01da02bf-f2d7-4d85-8e76-fe0e6a950301>

In [9]:
(job.ended-job.started).total_seconds()

31.818

### Check out this table in BigQuery Console:
- Click: https://console.cloud.google.com/bigquery
- Make sure project selected is the one from this notebook
- Under Explore, expand this project and review the dataset and table

---
## Review Data
Additional SQL queries could be used to review the data.  This section shows moving the table to a Pandas dataframe for local review in Python:

In [ ]:
#query = f"""
#SELECT *
#FROM `{DATANAME}.{DATANAME}`
#"""
#df = bq.query(query = query).to_dataframe()

---
## Prepare Data for Analysis

Create a prepped version of the data with test/train splits using SQL DDL:

In [6]:
query = f"""
CREATE OR REPLACE TABLE `{DATANAME}.{DATANAME}_notnull` AS
SELECT
  *
FROM
  `{DATANAME}.{DATANAME}`
WHERE
  tips IS NOT NULL
  AND trip_seconds IS NOT NULL
  AND trip_miles IS NOT NULL
  AND fare IS NOT NULL
  AND tolls IS NOT NULL
  AND pickup_location IS NOT NULL
  AND dropoff_location IS NOT NULL
  AND pickup_latitude IS NOT NULL
  AND pickup_longitude IS NOT NULL
  AND dropoff_latitude IS NOT NULL
  AND dropoff_longitude IS NOT NULL
  AND company IS NOT NULL
  AND trip_miles > 1
  AND trip_seconds > 180
"""
job = bq.query(query = query)
job.result()

In [7]:
(job.ended-job.started).total_seconds()

16.921

In [8]:
job.estimated_bytes_processed/1000000 #MB

73020.187269

Review the test/train split:

In [9]:
query = f"""
SELECT payment_type, count(*) as Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
FROM `{DATANAME}.{DATANAME}_notnull`
GROUP BY payment_type
"""
bq.query(query = query).to_dataframe()

,payment_type,Count,Percentage
0,Credit Card,26052429,46.395976
1,Pcard,7018,0.012498
2,Prcard,942643,1.678724
3,No Charge,276520,0.492446
4,Unknown,466041,0.829958
5,Dispute,33859,0.060298
6,Way2ride,75,0.000134
7,Prepaid,1267,0.002256
8,Cash,27759141,49.435407
9,Split,2263,0.004030


Retrieve a subset of the data to a Pandas dataframe:

In [10]:
query = f"""
SELECT * 
FROM `{DATANAME}.{DATANAME}_notnull`
LIMIT 5
"""
data = bq.query(query = query).to_dataframe()

In [11]:
data.head()

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,90755b02fda6a5d8734e4c2d86e6d43182e1c2b5,b'\xe7\xd7\xdc\xf3m9\xed\xbew\xed\xad6\xd3\xad...,2019-12-24 06:30:00+00:00,2019-12-24 06:45:00+00:00,1080,2.6,NaN,NaN,14,14,...,0.0,14.50,Credit Card,Taxi Affiliation Services,41.968069,-87.721559,POINT (-87.7215590627 41.968069),41.968069,-87.721559063,POINT (-87.7215590627 41.968069)
1,926c8e1e8d65809692f516c3b0c2d7597fa50657,b'\xd7\xbeZ\xe7\xcd\xbd\xf5\xa7\xf7k\xc7<\xf3~...,2014-07-11 20:15:00+00:00,2014-07-11 20:30:00+00:00,480,1.1,NaN,NaN,14,14,...,0.0,6.25,Cash,Taxi Affiliation Services,41.968069,-87.721559,POINT (-87.7215590627 41.968069),41.968069,-87.721559063,POINT (-87.7215590627 41.968069)
2,4e2b46ec348f7a27e84f937e97d3abfce4380259,b'\x7f\xde\xb4q\xa7\xb7\xf1\xaf\x1f\xe1\xb7\xd...,2016-09-04 02:00:00+00:00,2016-09-04 02:15:00+00:00,932,28.8,NaN,NaN,14,14,...,1.0,12.50,Cash,Chicago Carriage Cab Corp,41.968069,-87.721559,POINT (-87.7215590627 41.968069),41.968069,-87.721559063,POINT (-87.7215590627 41.968069)
3,bf1ab4d268aea28e8f1699a1262f87e6d37ac1c1,b'\xef\xbf9s\x87\xba\xf5\xbe4q\xcd9\xe3\xb6\xb...,2016-12-27 15:00:00+00:00,2016-12-27 17:45:00+00:00,9840,18.4,NaN,NaN,14,14,...,0.0,67.50,Cash,Globe Taxi,41.968069,-87.721559,POINT (-87.7215590627 41.968069),41.968069,-87.721559063,POINT (-87.7215590627 41.968069)
4,41cc1fa35d470268a4c8c71dda1f7308acec39fb,b'm\xfd\x1b\xf7\xce\xdf\xe3V\xbcs~\xbckN\xf6u\...,2017-02-24 22:15:00+00:00,2017-02-24 22:15:00+00:00,420,2.0,NaN,NaN,14,14,...,0.0,8.25,Cash,Chicago Taxicab,41.968069,-87.721559,POINT (-87.7215590627 41.968069),41.968069,-87.721559063,POINT (-87.7215590627 41.968069)


---
## Remove Resources
see notebook "99 - Cleanup"